# Chapter 17
## Working with APIs

### Using a Web API

See <https://api.github.com/search/repositories?q=language:python&sort=stars>

#### Processing an API Response

In [1]:
import requests

url = 'https://api.github.com/search/repositories?q=language:python&sort=starts'
r = requests.get(url)
print('Status code:', r.status_code)

Status code:  200


In [3]:
response_dict = r.json()
print(response_dict.keys())

dict_keys(['total_count', 'incomplete_results', 'items'])


#### Working with the response dictionary

In [4]:
print('Total repositories: ', response_dict['total_count'])

repo_dicts = response_dict['items']
print('Repositories returned:', len(repo_dicts))

Total repositories:  2180553
Repositories returned: 30


In [6]:
repo_dict = repo_dicts[0]
print('\nKeys:', len(repo_dict))
for key in sorted(repo_dict.keys()):
    print(key)


Keys: 72
archive_url
archived
assignees_url
blobs_url
branches_url
clone_url
collaborators_url
comments_url
commits_url
compare_url
contents_url
contributors_url
created_at
default_branch
deployments_url
description
downloads_url
events_url
fork
forks
forks_count
forks_url
full_name
git_commits_url
git_refs_url
git_tags_url
git_url
has_downloads
has_issues
has_pages
has_projects
has_wiki
homepage
hooks_url
html_url
id
issue_comment_url
issue_events_url
issues_url
keys_url
labels_url
language
languages_url
license
merges_url
milestones_url
mirror_url
name
notifications_url
open_issues
open_issues_count
owner
private
pulls_url
pushed_at
releases_url
score
size
ssh_url
stargazers_count
stargazers_url
statuses_url
subscribers_url
subscription_url
svn_url
tags_url
teams_url
trees_url
updated_at
url
watchers
watchers_count


In [10]:
print('\nSelected information about first repository:')
print('Name:', repo_dict['name'])
print('Owner:', repo_dict['owner']['login'])
print('Stars:', repo_dict['stargazers_count'])
print('Repository:', repo_dict['html_url'])
print('Created:', repo_dict['created_at'])
print('Updated:', repo_dict['updated_at'])
print('Description:', repo_dict['description'])


Selected information about first repository:
Name: awesome-python
Owner: vinta
Stars: 42210
Repository: https://github.com/vinta/awesome-python
Created: 2014-06-27T21:00:06Z
Updated: 2017-12-11T00:16:30Z
Description: A curated list of awesome Python frameworks, libraries, software and resources


#### Summarizing the top repositories

In [12]:
print('\nSelected information about each repository:')
for repo_dict in repo_dicts:
    print('\nName:', repo_dict['name'])
    print('Owner:', repo_dict['owner']['login'])
    print('Stars:', repo_dict['stargazers_count'])
    print('Repository:', repo_dict['html_url'])
    print('Created:', repo_dict['created_at'])
    print('Updated:', repo_dict['updated_at'])
    print('Description:', repo_dict['description'])


Selected information about each repository:

Name: awesome-python
Owner: vinta
Stars: 42210
Repository: https://github.com/vinta/awesome-python
Created: 2014-06-27T21:00:06Z
Updated: 2017-12-11T00:16:30Z
Description: A curated list of awesome Python frameworks, libraries, software and resources

Name: httpie
Owner: jakubroztocil
Stars: 32963
Repository: https://github.com/jakubroztocil/httpie
Created: 2012-02-25T12:39:13Z
Updated: 2017-12-11T01:02:15Z
Description: Modern command line HTTP client – user-friendly curl alternative with intuitive UI, JSON support, syntax highlighting, wget-like downloads, extensions, etc.  https://httpie.org

Name: thefuck
Owner: nvbn
Stars: 32514
Repository: https://github.com/nvbn/thefuck
Created: 2015-04-08T15:08:04Z
Updated: 2017-12-11T01:34:53Z
Description: Magnificent app which corrects your previous console command.

Name: youtube-dl
Owner: rg3
Stars: 31863
Repository: https://github.com/rg3/youtube-dl
Created: 2010-10-31T14:35:07Z
Updated: 2017-12

#### Monitoring API rate limits

See <https://api.github.com/rate_limit> to see your rate limit info.

### Visualizing repositories using Pygal

#### Refinging Pygal charts

In [30]:
import pygal
from pygal.style import LightColorizedStyle as LCS, LightenStyle as LS

names, stars = [], []
for repo_dict in repo_dicts:
    names.append(repo_dict['name'])
    stars.append(repo_dict['stargazers_count'])

style = LS('#333366', base_style=LCS)
style.title_font_size = 24
style.label_font_size = 14
style.major_label_font_size = 18

config = pygal.Config()
config.x_label_rotation = 45
config.show_legend = False
config.truncate_label = 15
config.show_y_guides = False
config.width = 1000

chart=pygal.Bar(style=style, config=config)
chart.title = 'Most-Starred Python Projects on GitHub'
chart.x_labels = names
chart.add('', stars)
chart.render_to_file('chapter_17_output/python_repos.svg')

#### Adding custom tooltips

In [31]:
chart = pygal.Bar(style=style, config=config)
chart.title = 'Python Projects'
chart.x_labels = ['httpie', 'django', 'flask']

plot_dicts = [
    {'value': 16101, 'label': 'Description of httpie.'},
    {'value': 15028, 'label': 'Description of django..'},
    {'value': 14798, 'label': 'Description of flask.'},
]

chart.add('', plot_dicts)
chart.render_to_file('chapter_17_output/bar_descriptions.svg')

#### Plotting the data

In [37]:
chart = pygal.Bar(style=style, config=config)
chart.title = 'Most-Starred Python Projects on GitHub'

names, plot_dicts = [], []
for repo_dict in repo_dicts:
    names.append(repo_dict['name'])
    description = repo_dict['description']
    if not description:
        description = 'No description provided.'
    
    plot_dicts.append({
        'value': repo_dict['stargazers_count'],
        'label': description,
    })

chart.x_labels = names
chart.add('', plot_dicts)
chart.render_to_file('chapter_17_output/python_repos_with_descriptions.svg')

#### Adding clickable links to our graph

In [38]:
chart = pygal.Bar(style=style, config=config)
chart.title = 'Most-Starred Python Projects on GitHub'

names, plot_dicts = [], []
for repo_dict in repo_dicts:
    names.append(repo_dict['name'])
    description = repo_dict['description']
    if not description:
        description = 'No description provided.'
    
    plot_dicts.append({
        'value': repo_dict['stargazers_count'],
        'label': description,
        'xlink': repo_dict['html_url']
    })

chart.x_labels = names
chart.add('', plot_dicts)
chart.render_to_file('chapter_17_output/python_repos_with_descriptions_and_links.svg')

### The Hacker News API

In [43]:
import requests

from operator import itemgetter

url_prefix = 'https://hacker-news.firebaseio.com/v0/'
url = url_prefix + 'topstories.json'
r = requests.get(url)
print('Status code:', r.status_code)

submission_ids = r.json()
submission_dicts = []
for submission_id in submission_ids[:30]:
    url = url_prefix + 'item/' + str(submission_id) + '.json'
    submission_request = requests.get(url)
    print('Status code:', submission_request.status_code)
    response_dict = submission_request.json()
    
    submission_dicts.append({
        'title': response_dict['title'],
        'link': 'http://news.ycombinator.com/item?id=' + str(submission_id),
        'comments': response_dict.get('descendants', 0)
    })

submission_dicts = sorted(submission_dicts, key=itemgetter('comments'), reverse=True)

print('len', len(submission_dicts))
for submission_dict in submission_dicts:
    print('\nTitle:', submission_dict['title'])
    print('Discussion link:', submission_dict['link'])
    print('Comments:', submission_dict['comments'])

Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
len 30

Title: Show HN: Airmash – Multiplayer Missile Warfare HTML5 Game
Discussion link: http://news.ycombinator.com/item?id=15892066
Comments: 191

Title: A beginner’s guide to getting started in the cryptocurrency world
Discussion link: http://news.ycombinator.com/item?id=15891344
Comments: 166

Title: Facebook wants your children's mind
Discussion link: http://news.ycombinator.com/item?id=15893834
Comments: 80

Title: What Makes a Great Software Engineer? [pdf]
Dis

#### Try it yourself

In [55]:
# 17-1. Other languages: Modify the API call in python_repos.py so it
# generates a chart showing the most popular projects in other languages.
# Try languages such as JavaScript, Ruby, C, Java, Perl, haskell and Go.

for language in ['js', 'ruby', 'c', 'java', 'perl', 'haskell', 'go']:
    url = 'https://api.github.com/search/repositories?q=language:' + language + '&sort=starts'
    r = requests.get(url)
    response_dict = r.json()
    repo_dicts = response_dict['items']

    chart = pygal.Bar(style=style, config=config)
    chart.title = 'Most-Starred ' + language.title() + ' Projects on GitHub'

    names, plot_dicts = [], []
    for repo_dict in repo_dicts:
        names.append(repo_dict['name'])
        description = repo_dict['description']
        if not description:
            description = 'No description provided.'

        plot_dicts.append({
            'value': repo_dict['stargazers_count'],
            'label': description,
            'xlink': repo_dict['html_url']
        })

    chart.x_labels = names
    chart.add('', plot_dicts)
    chart.render_to_file('chapter_17_output/' + language + '_repos_with_descriptions_and_links.svg')

In [54]:
# 17-2. Active discussions: Using the data from hn_submissions.py, make a bar
# chart showing the most active discussions currently happening on Hacker News.
# The height of each bar should correspond to the number of comments each
# submission has. The label for each bar should include the submission's title,
# and each bar should act as a link to the discussion page for that submission.

chart = pygal.Bar(style=style, config=config)
chart.title = 'Most active discussions currently on Hacker News'

titles, plot_dicts = [], []
for submission_dict in submission_dicts:
    title = submission_dict['title']
    titles.append(title)
    plot_dicts.append({
        'value': submission_dict['comments'],
        'label': title,
        'xlink': submission_dict['link']
    })

chart.x_labels = titles
chart.add('', plot_dicts)
chart.render_to_file('chapter_17_output/most_active_discussions_on_hn.svg')